In [1]:
import numpy as np
import scipy.sparse as sp
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

class LorenzSystem:
    def __init__(self, sigma, rho, beta, dt):
        self.sigma = sigma
        self.rho = rho
        self.beta = beta
        self.dt = dt
        self.state = np.array([-7.45, -3.03, 0.01])

    def step(self, fx=0, fy=0, fz=0):
        x, y, z = self.state
        xp = self.sigma * (y - x) + fx
        yp = x * (self.rho - z) - y + fy
        zp = x * y - self.beta * z + fz
        self.state += self.dt * np.array([xp, yp, zp])
        return self.state

class Reservoir:
    def __init__(self, n, m, mu, lam):
        self.n = n
        self.m = m
        self.lam = lam
        self.r = 0.5 - np.random.rand(n)
        self.Win = 0.01 * (2.0 * np.random.rand(n, m) - 1)
        self.A = self.create_adjacency_matrix(n, mu)

    def create_adjacency_matrix(self, n, mu):
        A = sp.random(n, n, density=6/n, format='csr')
        A = A - 0.5 * np.sign(A.toarray())
        A = A * mu / np.abs(sp.linalg.eigs(A, k=1)[0])
        return sp.csr_matrix(A)

    def update(self, control_input):
        self.r = np.tanh(self.A.dot(self.r) + self.Win.dot(control_input) + 1)
        return self.r

class Network:
    def __init__(self, n, m, mu, lam, sigma, rho, beta, dt, T):
        self.lorenz = LorenzSystem(sigma, rho, beta, dt)
        self.reservoir = Reservoir(n, m, mu, lam)
        self.T = T
        self.dt = dt
        self.ntraining = int(T / dt)
        self.R = np.zeros((n, self.ntraining))
        self.f = np.zeros((m, self.ntraining))

    def train(self):
        for t in range(self.ntraining):
            fx = np.cos(0.05 * self.dt * t)
            fy = np.sin(0.05 * self.dt * t)
            fz = 0  # Default value for fz

            self.R[:, t] = self.reservoir.r
            self.f[0, t] = fx
            self.f[1, t] = fy

            self.lorenz.step(fx, fy, fz)
            control_input = np.array(self.lorenz.state)
            self.reservoir.update(control_input)

        Wout = self.f.dot(self.R.T).dot(np.linalg.inv(self.R.dot(self.R.T) + self.reservoir.lam * np.eye(self.reservoir.n)))
        return Wout

class Controller:
    def __init__(self, ntest, alpha, tau, Wout, reservoir):
        self.ntest = ntest
        self.alpha = alpha
        self.tau = tau
        self.Wout = Wout
        self.reservoir = reservoir
        self.v = np.zeros((3, ntest + 1))
        self.control_states = [np.zeros(ntest) for _ in range(3)]
        self.nocontrol_states = [np.zeros(ntest) for _ in range(3)]
        self.pure_states = [np.zeros(ntest) for _ in range(3)]
        self.temp = Wout.dot(reservoir.r)

    def run(self):
        for t in range(self.ntest):
            # Pure system
            self.update_pure(t)
            # No control
            self.update_no_control(t)
            # With control
            self.update_with_control(t)

    def update_pure(self, t):
        x, y, z = (0, 0, 0) if t == 0 else self.pure_states[:, t - 1]
        fx, fy, fz = 0, 0, 0  # No external force
        new_state = self.reservoir.update(np.array([x, y, z])) + np.array([fx, fy, fz])
        self.pure_states[:, t] = new_state

    def update_no_control(self, t):
        x, y, z = (0, 0, 0) if t == 0 else self.nocontrol_states[:, t - 1]
        gx = 20 * self.nocontrol_states[0, t - 1] if t > 0 else 0
        new_state = self.reservoir.update(np.array([x, y, z])) + np.array([gx, 0, 0])  # Apply disturbance
        self.nocontrol_states[:, t] = new_state

    def update_with_control(self, t):
        gx = 20 * self.nocontrol_states[0, t - 1] if t > 0 else 0
        ux = self.temp[0]
        self.v[:, t + 1] = self.v[:, t] + (self.dt / self.tau) * (self.temp - self.v[:, t])
        
        x, y, z = self.nocontrol_states[:, t]
        new_state = self.reservoir.update(np.array([x, y, z])) + np.array([gx, 0, 0]) - self.alpha * self.v[:, t]
        self.control_states[:, t] = new_state
        self.temp = self.Wout.dot(new_state)

    def get_states(self):
        return self.pure_states, self.nocontrol_states, self.control_states

class Plotter:
    @staticmethod
    def plot_3d(x, y, z, color='b', label=None):
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        ax.plot3D(x, y, z, color=color, linewidth=0.1, label=label)
        if label:
            ax.legend()
        plt.show()

def main():
    # Parameters
    sigma = 10
    rho = 28
    beta = 8/3
    n = 1000
    m = 3
    mu = 1.2
    lam = 0.000001
    alpha = 100
    tau = 10
    T = 100
    dt = 0.002
    ntraining = int(T / dt)

    network = Network(n, m, mu, lam, sigma, rho, beta, dt, T)
    Wout = network.train()

    ntest = int(200 / dt)
    controller = Controller(ntest, alpha, tau, Wout, network.reservoir)
    controller.run()
    pure_states, nocontrol_states, control_states = controller.get_states()

    Plotter.plot_3d(pure_states[0], pure_states[1], pure_states[2], color='r', label='Pure')
    Plotter.plot_3d(nocontrol_states[0], nocontrol_states[1], nocontrol_states[2], color='b', label='No Control')
    Plotter.plot_3d(control_states[0], control_states[1], control_states[2], color='g', label='With Control')

if __name__ == "__main__":
    main()


ValueError: operands could not be broadcast together with shapes (1000,) (3,) 

In [2]:
import numpy as np
import scipy.sparse as sp
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

class LorenzSystem:
    def __init__(self, sigma, rho, beta, dt):
        self.sigma = sigma
        self.rho = rho
        self.beta = beta
        self.dt = dt
        self.state = np.array([-7.45, -3.03, 0.01])

    def step(self, fx=0, fy=0, fz=0):
        x, y, z = self.state
        xp = self.sigma * (y - x) + fx
        yp = x * (self.rho - z) - y + fy
        zp = x * y - self.beta * z + fz
        self.state += self.dt * np.array([xp, yp, zp])
        return self.state

class Reservoir:
    def __init__(self, n, m, mu, lam):
        self.n = n
        self.m = m
        self.lam = lam
        self.r = 0.5 - np.random.rand(n)
        self.Win = 0.01 * (2.0 * np.random.rand(n, m) - 1)
        self.A = self.create_adjacency_matrix(n, mu)

    def create_adjacency_matrix(self, n, mu):
        A = sp.random(n, n, density=6/n, format='csr')
        A = A - 0.5 * np.sign(A.toarray())
        A = A * mu / np.abs(sp.linalg.eigs(A, k=1)[0])
        return sp.csr_matrix(A)

    def update(self, control_input):
        self.r = np.tanh(self.A.dot(self.r) + self.Win.dot(control_input) + 1)
        return self.r

class Network:
    def __init__(self, n, m, mu, lam, sigma, rho, beta, dt, T):
        self.lorenz = LorenzSystem(sigma, rho, beta, dt)
        self.reservoir = Reservoir(n, m, mu, lam)
        self.T = T
        self.dt = dt
        self.ntraining = int(T / dt)
        self.R = np.zeros((n, self.ntraining))
        self.f = np.zeros((m, self.ntraining))

    def train(self):
        for t in range(self.ntraining):
            fx = np.cos(0.05 * self.dt * t)
            fy = np.sin(0.05 * self.dt * t)
            fz = 0  # Default value for fz

            self.R[:, t] = self.reservoir.r
            self.f[0, t] = fx
            self.f[1, t] = fy

            self.lorenz.step(fx, fy, fz)
            control_input = self.lorenz.state  # Use the updated Lorenz state
            self.reservoir.update(control_input)

        Wout = self.f.dot(self.R.T).dot(np.linalg.inv(self.R.dot(self.R.T) + self.reservoir.lam * np.eye(self.reservoir.n)))
        return Wout

class Controller:
    def __init__(self, ntest, alpha, tau, Wout, reservoir):
        self.ntest = ntest
        self.alpha = alpha
        self.tau = tau
        self.Wout = Wout
        self.reservoir = reservoir
        self.v = np.zeros((3, ntest + 1))
        self.control_states = [np.zeros(ntest) for _ in range(3)]
        self.nocontrol_states = [np.zeros(ntest) for _ in range(3)]
        self.pure_states = [np.zeros(ntest) for _ in range(3)]
        self.temp = Wout.dot(reservoir.r)

    def run(self):
        for t in range(self.ntest):
            # Pure system
            self.update_pure(t)
            # No control
            self.update_no_control(t)
            # With control
            self.update_with_control(t)

    def update_pure(self, t):
        x, y, z = (0, 0, 0) if t == 0 else self.pure_states[:, t - 1]
        fx, fy, fz = 0, 0, 0  # No external force
        new_state = self.reservoir.update(np.array([x, y, z]))  # No force applied here
        self.pure_states[:, t] = new_state

    def update_no_control(self, t):
        x, y, z = (0, 0, 0) if t == 0 else self.nocontrol_states[:, t - 1]
        gx = 20 * self.nocontrol_states[0, t - 1] if t > 0 else 0
        new_state = self.reservoir.update(np.array([x, y, z])) + np.array([gx, 0, 0])  # Apply disturbance
        self.nocontrol_states[:, t] = new_state

    def update_with_control(self, t):
        gx = 20 * self.nocontrol_states[0, t - 1] if t > 0 else 0
        ux = self.temp[0]
        self.v[:, t + 1] = self.v[:, t] + (self.dt / self.tau) * (self.temp - self.v[:, t])
        
        x, y, z = self.nocontrol_states[:, t]
        new_state = self.reservoir.update(np.array([x, y, z])) + np.array([gx, 0, 0]) - self.alpha * self.v[:, t]
        self.control_states[:, t] = new_state
        self.temp = self.Wout.dot(new_state)

    def get_states(self):
        return self.pure_states, self.nocontrol_states, self.control_states

class Plotter:
    @staticmethod
    def plot_3d(x, y, z, color='b', label=None):
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        ax.plot3D(x, y, z, color=color, linewidth=0.1, label=label)
        if label:
            ax.legend()
        plt.show()

def main():
    # Parameters
    sigma = 10
    rho = 28
    beta = 8/3
    n = 1000
    m = 3
    mu = 1.2
    lam = 0.000001
    alpha = 100
    tau = 10
    T = 100
    dt = 0.002
    ntraining = int(T / dt)

    network = Network(n, m, mu, lam, sigma, rho, beta, dt, T)
    Wout = network.train()

    ntest = int(200 / dt)
    controller = Controller(ntest, alpha, tau, Wout, network.reservoir)
    controller.run()
    pure_states, nocontrol_states, control_states = controller.get_states()

    Plotter.plot_3d(pure_states[0], pure_states[1], pure_states[2], color='r', label='Pure')
    Plotter.plot_3d(nocontrol_states[0], nocontrol_states[1], nocontrol_states[2], color='b', label='No Control')
    Plotter.plot_3d(control_states[0], control_states[1], control_states[2], color='g', label='With Control')

if __name__ == "__main__":
    main()


TypeError: list indices must be integers or slices, not tuple

In [3]:
import numpy as np
import scipy.sparse as sp
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

class LorenzSystem:
    def __init__(self, sigma, rho, beta, dt):
        self.sigma = sigma
        self.rho = rho
        self.beta = beta
        self.dt = dt
        self.state = np.array([-7.45, -3.03, 0.01])

    def step(self, fx=0, fy=0, fz=0):
        x, y, z = self.state
        xp = self.sigma * (y - x) + fx
        yp = x * (self.rho - z) - y + fy
        zp = x * y - self.beta * z + fz
        self.state += self.dt * np.array([xp, yp, zp])
        return self.state

class Reservoir:
    def __init__(self, n, m, mu, lam):
        self.n = n
        self.m = m
        self.lam = lam
        self.r = 0.5 - np.random.rand(n)
        self.Win = 0.01 * (2.0 * np.random.rand(n, m) - 1)
        self.A = self.create_adjacency_matrix(n, mu)

    def create_adjacency_matrix(self, n, mu):
        A = sp.random(n, n, density=6/n, format='csr')
        A = A - 0.5 * np.sign(A.toarray())
        A = A * mu / np.abs(sp.linalg.eigs(A, k=1)[0])
        return sp.csr_matrix(A)

    def update(self, control_input):
        self.r = np.tanh(self.A.dot(self.r) + self.Win.dot(control_input) + 1)
        return self.r

class Network:
    def __init__(self, n, m, mu, lam, sigma, rho, beta, dt, T):
        self.lorenz = LorenzSystem(sigma, rho, beta, dt)
        self.reservoir = Reservoir(n, m, mu, lam)
        self.T = T
        self.dt = dt
        self.ntraining = int(T / dt)
        self.R = np.zeros((n, self.ntraining))
        self.f = np.zeros((m, self.ntraining))

    def train(self):
        for t in range(self.ntraining):
            fx = np.cos(0.05 * self.dt * t)
            fy = np.sin(0.05 * self.dt * t)
            fz = 0  # Default value for fz

            self.R[:, t] = self.reservoir.r
            self.f[0, t] = fx
            self.f[1, t] = fy

            self.lorenz.step(fx, fy, fz)
            control_input = self.lorenz.state  # Use the updated Lorenz state
            self.reservoir.update(control_input)

        Wout = self.f.dot(self.R.T).dot(np.linalg.inv(self.R.dot(self.R.T) + self.reservoir.lam * np.eye(self.reservoir.n)))
        return Wout

class Controller:
    def __init__(self, ntest, alpha, tau, Wout, reservoir):
        self.ntest = ntest
        self.alpha = alpha
        self.tau = tau
        self.Wout = Wout
        self.reservoir = reservoir
        self.v = np.zeros((3, ntest + 1))
        self.control_states = np.zeros((3, ntest))  # Correctly initialize as a 2D array
        self.nocontrol_states = np.zeros((3, ntest))  # Correctly initialize as a 2D array
        self.pure_states = np.zeros((3, ntest))  # Correctly initialize as a 2D array
        self.temp = Wout.dot(reservoir.r)

    def run(self):
        for t in range(self.ntest):
            # Pure system
            self.update_pure(t)
            # No control
            self.update_no_control(t)
            # With control
            self.update_with_control(t)

    def update_pure(self, t):
        x, y, z = (0, 0, 0) if t == 0 else self.pure_states[:, t - 1]
        fx, fy, fz = 0, 0, 0  # No external force
        new_state = self.reservoir.update(np.array([x, y, z]))  # No force applied here
        self.pure_states[:, t] = new_state  # Store new state correctly

    def update_no_control(self, t):
        x, y, z = (0, 0, 0) if t == 0 else self.nocontrol_states[:, t - 1]
        gx = 20 * self.nocontrol_states[0, t - 1] if t > 0 else 0
        new_state = self.reservoir.update(np.array([x, y, z])) + np.array([gx, 0, 0])  # Apply disturbance
        self.nocontrol_states[:, t] = new_state  # Store new state correctly

    def update_with_control(self, t):
        gx = 20 * self.nocontrol_states[0, t - 1] if t > 0 else 0
        ux = self.temp[0]
        self.v[:, t + 1] = self.v[:, t] + (self.dt / self.tau) * (self.temp - self.v[:, t])
        
        x, y, z = self.nocontrol_states[:, t]
        new_state = self.reservoir.update(np.array([x, y, z])) + np.array([gx, 0, 0]) - self.alpha * self.v[:, t]
        self.control_states[:, t] = new_state  # Store new state correctly
        self.temp = self.Wout.dot(new_state)

    def get_states(self):
        return self.pure_states, self.nocontrol_states, self.control_states

class Plotter:
    @staticmethod
    def plot_3d(x, y, z, color='b', label=None):
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        ax.plot3D(x, y, z, color=color, linewidth=0.1, label=label)
        if label:
            ax.legend()
        plt.show()

def main():
    # Parameters
    sigma = 10
    rho = 28
    beta = 8/3
    n = 1000
    m = 3
    mu = 1.2
    lam = 0.000001
    alpha = 100
    tau = 10
    T = 100
    dt = 0.002
    ntraining = int(T / dt)

    network = Network(n, m, mu, lam, sigma, rho, beta, dt, T)
    Wout = network.train()

    ntest = int(200 / dt)
    controller = Controller(ntest, alpha, tau, Wout, network.reservoir)
    controller.run()
    pure_states, nocontrol_states, control_states = controller.get_states()

    Plotter.plot_3d(pure_states[0], pure_states[1], pure_states[2], color='r', label='Pure')
    Plotter.plot_3d(nocontrol_states[0], nocontrol_states[1], nocontrol_states[2], color='b', label='No Control')
    Plotter.plot_3d(control_states[0], control_states[1], control_states[2], color='g', label='With Control')

if __name__ == "__main__":
    main()


ValueError: could not broadcast input array from shape (1000,) into shape (3,)

In [6]:
import numpy as np
import scipy.sparse as sp
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

class LorenzSystem:
    def __init__(self, sigma, rho, beta, dt):
        self.sigma = sigma
        self.rho = rho
        self.beta = beta
        self.dt = dt
        self.state = np.array([-7.45, -3.03, 0.01])

    def step(self, fx=0, fy=0, fz=0):
        x, y, z = self.state
        xp = self.sigma * (y - x) + fx
        yp = x * (self.rho - z) - y + fy
        zp = x * y - self.beta * z + fz
        self.state += self.dt * np.array([xp, yp, zp])
        return self.state

class Reservoir:
    def __init__(self, n, m, mu, lam):
        self.n = n
        self.m = m
        self.lam = lam
        self.r = 0.5 - np.random.rand(n)
        self.Win = 0.01 * (2.0 * np.random.rand(n, m) - 1)
        self.A = self.create_adjacency_matrix(n, mu)

    def create_adjacency_matrix(self, n, mu):
        A = sp.random(n, n, density=6/n, format='csr')
        A = A - 0.5 * np.sign(A.toarray())
        A = A * mu / np.abs(sp.linalg.eigs(A, k=1)[0])
        return sp.csr_matrix(A)

    def update(self, control_input):
        self.r = np.tanh(self.A.dot(self.r) + self.Win.dot(control_input) + 1)
        return self.r

class Network:
    def __init__(self, n, m, mu, lam, sigma, rho, beta, dt, T):
        self.lorenz = LorenzSystem(sigma, rho, beta, dt)
        self.reservoir = Reservoir(n, m, mu, lam)
        self.Tf = T
        self.dt = dt
        self.ntraining = int(T / dt)
        self.R = np.zeros((n, self.ntraining))
        self.f = np.zeros((m, self.ntraining))
        self.Wout = np.zeros((m, n))

    def train(self):
        for t in range(self.ntraining):
            
            control_input = self.lorenz.state  # Use the updated Lorenz state
            self.reservoir.update(control_input)

            fx = np.cos(0.05 * self.dt * t)
            fy = np.sin(0.05 * self.dt * t)
            fz = 0  # Default value for fz

            self.R[:, t] = self.reservoir.r
            self.f[0, t] = fx
            self.f[1, t] = fy
            self.f[2, t] = fz

            self.lorenz.step(fx, fy, fz)
            

        self.Wout = self.f.dot(self.R.T).dot(np.linalg.inv(self.R.dot(self.R.T) + self.reservoir.lam * np.eye(self.reservoir.n)))


class Evolution:
    def __init__(self, Ttest, alpha, tau, Wout, reservoir, dt):
        self.Tf = Ttest
        self.ntest = int(self.Tf/self.dt)
        self.alpha = alpha
        self.tau = tau
        self.Wout = Wout
        self.reservoir = reservoir
        self.dt = dt  # Set dt here
        self.v = np.zeros((3, ntest + 1))
        self.control_states = np.zeros((3, ntest))  # Store x, y, z for control
        self.nocontrol_states = np.zeros((3, ntest))  # Store x, y, z for no control
        self.pure_states = np.zeros((3, ntest))  # Store x, y, z for pure
        self.temp = Wout.dot(reservoir.r)

    def run(self):
        for t in range(self.ntest):
            # Pure system
            self.update_pure(t)
            # No control
            self.update_no_control(t)
            # With control
            self.update_with_control(t)

    def update_pure(self, t):
        new_state = self.reservoir.update(np.array([0, 0, 0]))  # No force applied
        self.pure_states[:, t] = new_state[:3]  # Store only x, y, z

    def update_no_control(self, t):
        x, y, z = (0, 0, 0) if t == 0 else self.nocontrol_states[:, t - 1]
        gx = 20 * self.nocontrol_states[0, t - 1] if t > 0 else 0
        
        # Update reservoir with current state
        new_state = self.reservoir.update(np.array([x, y, z]))  # Output is of shape (1000,)
        
        # Extract the first three components for x, y, z
        self.nocontrol_states[:, t] = new_state[:3] + np.array([gx, 0, 0])  # Store only x, y, z

    def update_with_control(self, t):
        gx = 20 * self.nocontrol_states[0, t - 1] if t > 0 else 0
        
        ux = self.temp[0]
        self.v[:, t + 1] = self.v[:, t] + (self.dt / self.tau) * (self.temp - self.v[:, t])

        x, y, z = self.nocontrol_states[:, t]
        
        # Update reservoir with current state
        new_state = self.reservoir.update(np.array([x, y, z]))  # Output is of shape (1000,)
        
        # Calculate new control state and store x, y, z
        self.control_states[:, t] = new_state[:3] + np.array([gx, 0, 0]) - self.alpha * self.v[:, t]
        self.temp = self.Wout.dot(self.control_states[:, t])  # Update temp for next iteration

    def get_states(self):
        return self.pure_states, self.nocontrol_states, self.control_states


class Plotter:
    @staticmethod
    def plot_3d(x, y, z, color='b', label=None):
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        ax.plot3D(x, y, z, color=color, linewidth=0.1, label=label)
        if label:
            ax.legend()
        plt.show()

def main():
    # Parameters
    sigma = 10
    rho = 28
    beta = 8/3
    n = 1000
    m = 3
    mu = 1.2
    lam = 0.000001
    
    Ttrain = 100
    dt = 0.002

    network = Network(n, m, mu, lam, sigma, rho, beta, dt, Ttrain)
    network.train()

    alpha = 100
    tau = 10

    Ttest = 200
    controller = Evolution(Ttest, alpha, tau, network, dt)  # Pass dt here
    controller.run()
    pure_states, nocontrol_states, control_states = controller.get_states()

    Plotter.plot_3d(pure_states[0], pure_states[1], pure_states[2], color='r', label='Pure')
    Plotter.plot_3d(nocontrol_states[0], nocontrol_states[1], nocontrol_states[2], color='b', label='No Control')
    Plotter.plot_3d(control_states[0], control_states[1], control_states[2], color='g', label='With Control')

if __name__ == "__main__":
    main()


ValueError: shapes (3,1000) and (3,) not aligned: 1000 (dim 1) != 3 (dim 0)